In [60]:
## Installing Required Packages
import sys
#!{sys.executable} -m pip install alpaca_trade_api
#!{sys.executable} -m pip install tqdm
#!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install ta

  Using cached ta-0.5.25.tar.gz (20 kB)
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24884 sha256=d893933d9d73c04b887dca183e4c3068c2b757997560920f3cfc75f27c988844
  Stored in directory: c:\users\aayorde\appdata\local\pip\cache\wheels\da\58\47\e9a2cbcf4374b34f18c4207eeab57571849f3c9dc858848e89
Successfully built ta


In [39]:
## Loading Packages
import alpaca_trade_api as tradeapi
import os
import pandas
from datetime import datetime, timedelta

Project_Folder = "C://Users//" + os.getlogin() + "//documents//github//Stock-Strategy-Exploration//"

N_DAYS_AGO = 365*3
start_date = datetime.now() - timedelta(days=N_DAYS_AGO)
stock = "MSFT"

## Pulling API Keys
Keys = pandas.read_csv(os.path.join('..','Data','Keys','Live API.txt'))

## Connecting to Alpaca
api = tradeapi.REST(Keys['Key ID'][0],Keys['Secret Key'][0], api_version='v2')

## Testing API Functionality
### Get Current Account Information
account = api.get_account()
### List of Tradable Assets
assets = api.list_assets("active")
Assets = [i._raw['symbol'] for i in assets if i._raw['tradable'] & i._raw['shortable']  & i._raw['easy_to_borrow']]
### Pulling OHLC Data
bars = api.polygon.historic_agg_v2(stock, 1, 'day', _from=start_date.strftime("%Y-%m-%d"), to=datetime.now().strftime("%Y-%m-%d")).df



In [13]:
from collections import defaultdict
from tqdm.auto import tqdm
import pickle

## Amount of Historical Data to Pull
N_DAYS_AGO = 365*3

## Checking if Data Exists
if os.path.exists(Project_Folder + "Data//Historical_Data.p"):
    start_date = Historical_Data["MSFT"].index.max() + timedelta(days = 1)
    Historical_Data = pickle.load(open(Project_Folder + "Data//Historical_Data.p" , "rb" ))
else:
    start_date = datetime.now() - timedelta(days=N_DAYS_AGO)
    Historical_Data = defaultdict(pandas.DataFrame)

## Pulling Historical Data
for i in tqdm(Assets):
    try:
        Historical_Data[i] =  Historical_Data[i].append(api.polygon.historic_agg_v2(i, 1, 'day', _from=start_date.strftime("%Y-%m-%d"), to=datetime.now().strftime("%Y-%m-%d")).df, ignore_index=True)
    except:
        print(i + " : Failed To Pull Data")

## Saving Raw Data
pickle.dump(Historical_Data, open(Project_Folder + "Data//Historical_Data.p" , "wb" ) )

BMY.RTFailed To Pull Data



In [68]:
import ta

TMP = ta.add_all_ta_features(Historical_Data['MSFT'], open="open", high="high", low="low", close="close", volume="volume")


C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\ProgramData\Miniconda3\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [62]:
TMP = Historical_Data['MSFT']
ATR = ta.volatility.AverageTrueRange()

,open,high,low,close,volume
timestamp,,,,,
2017-07-13 00:00:00-04:00,71.5000,72.0399,71.31,71.77,20269816.0
2017-07-14 00:00:00-04:00,72.2400,73.2700,71.96,72.78,25868086.0
2017-07-17 00:00:00-04:00,72.8000,73.4500,72.72,73.35,21803907.0
2017-07-18 00:00:00-04:00,73.0900,73.3900,72.66,73.30,26435296.0
2017-07-19 00:00:00-04:00,73.5000,74.0400,73.45,73.86,22416222.0
...,...,...,...,...,...
2020-07-06 00:00:00-04:00,208.8335,211.1300,208.09,210.70,31897629.0
2020-07-07 00:00:00-04:00,210.4500,214.6700,207.99,208.25,33600708.0
2020-07-08 00:00:00-04:00,210.0700,213.2600,208.69,212.83,33600025.0
